<a href="https://colab.research.google.com/github/Gabrielghj/Bot-de-Trading-con-Backtrader---Versi-n-Mejorada/blob/main/Bot_de_Trading_con_Backtrader_Version_Mejorada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalar librerías necesarias
!pip install backtrader yfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 7.1 MB/s eta 0:00:00


In [2]:
#import backtrader as bt
import yfinance as yf
import pandas as pd

In [3]:
# Descargar datos históricos de Bitcoin desde Yahoo Finance
ticker = "BTC-USD"
data = yf.download(ticker, start="2023-01-01", end="2024-01-01")

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [4]:
# Mostrar las primeras filas de los datos para asegurarnos que se descargaron correctamente
print(data.head())

Price              Close          High           Low          Open  \
Ticker           BTC-USD       BTC-USD       BTC-USD       BTC-USD   
Date                                                                 
2023-01-01  16625.080078  16630.439453  16521.234375  16547.914062   
2023-01-02  16688.470703  16759.343750  16572.228516  16625.509766   
2023-01-03  16679.857422  16760.447266  16622.371094  16688.847656   
2023-01-04  16863.238281  16964.585938  16667.763672  16680.205078   
2023-01-05  16836.736328  16884.021484  16790.283203  16863.472656   

Price            Volume  
Ticker          BTC-USD  
Date                     
2023-01-01   9244361700  
2023-01-02  12097775227  
2023-01-03  13903079207  
2023-01-04  18421743322  
2023-01-05  13692758566  


In [5]:
# Guardar en CSV para simular la lectura desde un archivo (opcional)
data.to_csv("btc_data.csv")

In [6]:
# Crear un feed de datos para Backtrader
data_feed = bt.feeds.PandasData(dataname=data)

NameError: name 'bt' is not defined

In [ ]:
# Crear el cerebro de Backtrader
cerebro = bt.Cerebro()
cerebro.adddata(data_feed)

In [ ]:
# Configurar el capital inicial
cerebro.broker.setcash(10000.0)

In [ ]:
# Configurar el tamaño de la posición
cerebro.addsizer(bt.sizers.FixedSize, stake=1)

In [ ]:
# Configurar la comisión por operación
cerebro.broker.setcommission(commission=0.001)

In [ ]:
# Definir estrategia con RSI y Media Móvil
class RSIMovingAverageStrategy(bt.Strategy):
    def __init__(self):
        self.rsi = bt.indicators.RSI_SMA(self.data.close, period=14)
        self.sma = bt.indicators.SimpleMovingAverage(self.data.close, period=50)
        self.position_open = False

    def next(self):
        if not self.position_open and self.rsi[0] < 30 and self.data.close[0] > self.sma[0]:
            self.buy()
            self.position_open = True
            print(f"Compra a {self.data.close[0]} en {self.data.datetime.date(0)}")
        elif self.position_open and self.rsi[0] > 70:
            self.sell()
            self.position_open = False
            print(f"Venta a {self.data.close[0]} en {self.data.datetime.date(0)}")

In [ ]:
# Agregar la estrategia
cerebro.addstrategy(RSIMovingAverageStrategy)

In [ ]:
# Ejecutar la estrategia
cerebro.run()

In [ ]:
# Mostrar el valor final del portafolio
print(f'Valor final del portafolio: {cerebro.broker.getvalue()}')

In [ ]:
# Graficar los resultados
cerebro.plot()